In [1]:
import os
import pickle
import numpy as np
from numba import prange, njit
import importlib
import sys
import random
import threading
from multiprocessing import Process

In [2]:
import ModelConfig as mc
importlib.reload(mc)
import TensorflowModel as tm
importlib.reload(tm)
import ModelStrategies as ms
importlib.reload(ms)
import GameBoard as gb
importlib.reload(gb)


<module 'GameBoard' from 'c:\\Users\\akoko\\Documents\\Codes\\tic-tac-toe-ML\\tic-tac-toe-5-training\\GameBoard.py'>

In [3]:
# # print train position
# data = 4,0,2,8,6,9,3,1,3,4,9,2,[5, 7]
# board = [
# 	'  ', '  ', '  ',
# 	'  ', '  ', '  ',
# 	'  ', '  ', '  ',
# ]
# for i in range(9):
# 	index = i//2 + 1 if i % 2 == 0 else i//2 + 7
# 	if (data[index] == 9):
# 		for j in data[12]:
# 			board[j] = '}{' if i % 2 == 0 else '[]'
# 	else:
# 		board[data[index]] = '><' if i % 2 == 0 else '()'
# 	print(i + 1, 'ход')
# 	print(*board[:3], sep='│')
# 	print('──┼──┼──')
# 	print(*board[3:6], sep='│')
# 	print('──┼──┼──')
# 	print(*board[6:], sep='│')
# 	print()
# 	if (data[index] == 9):
# 		break

In [4]:
x_dataset = np.genfromtxt('x_train_dataset.csv', delimiter=',', skip_header=1, dtype=np.str_)
random.seed(5432)
test_len = int(len(x_dataset)*.1)
x_dataset = x_dataset.tolist()
random.shuffle(x_dataset)
x_test_dataset, x_train_dataset = x_dataset[:test_len], x_dataset[test_len:]
print(len(x_test_dataset), len(x_train_dataset))

o_dataset = np.genfromtxt('o_train_dataset.csv', delimiter=',', skip_header=1, dtype=np.str_)
test_len = int(len(o_dataset)*.1)
o_dataset = o_dataset.tolist()
random.shuffle(o_dataset)
o_test_dataset, o_train_dataset = o_dataset[:test_len], o_dataset[test_len:]
print(len(o_test_dataset), len(o_train_dataset))

1424 12820
2940 26460


In [5]:
models = []
for i in range(1, 11):
	config = mc.ModelConfig(
		hidden_structure=[1], 
		input_strategy_number=i, 
		output_strategy_number=1, 
		use_bias=True,
		use_attention=False,
		model_name=f'dataset_{i}'
	)
	t_model = tm.TensorflowModel(config)
	models.append(t_model)

In [6]:
# t_model = tm.TensorflowModel.fromFile('testModel')

In [7]:
def append_train_program(train_dataset, is_x, 
		x_train_moves, o_train_moves, allowed_train_moves, 
		correct_train_moves, train_as_x):
	count = len(train_dataset)
	for i in prange(count):
		data = train_dataset[i]
		x_moves = []
		o_moves = []
		allowed_moves = [
			0, 1, 2,
			3, 4, 5,
			6, 7, 8,
		]
		correct_moves = [int(x) for x in data[12].replace('[', '').replace(']', '').split(' ')]
		for x_i in range(int(data[0])):
			index = int(data[x_i + 1])
			x_moves.append(index)
		for o_i in range(int(data[6])):
			index = int(data[o_i + 7])
			o_moves.append(index)
		board = gb.GameBoard(x_moves, o_moves, allowed_moves, correct_moves)
		x_train_moves.append(board.x_mirrored_moves_array)
		o_train_moves.append(board.o_mirrored_moves_array)
		allowed_train_moves.append(board.allowed_mirrored_moves_array)
		correct_train_moves.append(board.correct_mirrored_moves_array)
		train_as_x.append(is_x)
		

In [8]:
x_train_moves = []
o_train_moves = []
allowed_train_moves = []
correct_train_moves = []
train_as_x = []
append_train_program(x_train_dataset, True,
		x_train_moves, o_train_moves, allowed_train_moves, 
		correct_train_moves, train_as_x)

append_train_program(o_train_dataset, False,
		x_train_moves, o_train_moves, allowed_train_moves, 
		correct_train_moves, train_as_x)

x_test_moves = []
o_test_moves = []
allowed_test_moves = []
correct_test_moves = []
test_as_x = []
append_train_program(x_test_dataset, True,
		x_test_moves, o_test_moves, allowed_test_moves, 
		correct_test_moves, test_as_x)

append_train_program(o_test_dataset, False,
		x_test_moves, o_test_moves, allowed_test_moves, 
		correct_test_moves, test_as_x)

print(len(x_test_moves))

for t_model in models:
	x_train, y_train = t_model.getModelDataset(
		x_train_moves,
		o_train_moves,
		allowed_train_moves,
		correct_train_moves,
		train_as_x
	)
	x_test, y_test = t_model.getModelDataset(
		x_test_moves,
		o_test_moves,
		allowed_test_moves,
		correct_test_moves,
		test_as_x
	)
	strategy = t_model.config.input_strategy_number
	with open(f'datasets/train_{strategy:02d}.pkl', 'wb') as file:
		pickle.dump([x_train, y_train], file)
	with open(f'datasets/test_{strategy:02d}.pkl', 'wb') as file:
		pickle.dump([x_test, y_test], file)



4364


c:\Users\akoko\Documents\Codes\tic-tac-toe-ML\tic-tac-toe-5-training\ModelStrategies.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to int8. Precision may be lost.
  inputs[0, i] = 0.99 if i in x_moves else 0.01


In [23]:
with open(f'datasets/train_01.pkl', 'rb') as file:
	x_train, y_train = pickle.load(file)
	board = [
		'  ', '  ', '  ',
		'  ', '  ', '  ',
		'  ', '  ', '  ',
	]
	train_index = 1
	inputs = x_train[train_index]
	outputs = y_train[train_index]
	print(inputs)
	print(inputs[:3])
	print(inputs[3:6])
	print(inputs[6:9])
	print()
	print(inputs[9:12])
	print(inputs[12:15])
	print(inputs[15:])
	print()
	print(outputs)
	print(outputs[:3])
	print(outputs[3:6])
	print(outputs[6:9])
	index = 0
	counter = 0
	for i in inputs[:9]:
		if (i == .99):
			board[index] = '><'
			counter += 1 
		index += 1
	index = 0
	for i in inputs[9:18]:
		if (i == .99):
			board[index] = '()'
			counter += 1 
		index += 1
	is_x = inputs[18] == .99
	index = 0
	for i in outputs:
		if (i == .99):
			board[index] = '}{' if is_x else '[]'
		index += 1

	print(*board[:3], sep='│')
	print('──┼──┼──')
	print(*board[3:6], sep='│')
	print('──┼──┼──')
	print(*board[6:], sep='│')
	print()

[0.01 0.01 0.01 0.99 0.01 0.99 0.01 0.99 0.01 0.01 0.99 0.01 0.01 0.99
 0.01 0.99 0.01 0.01 0.99]
[0.01 0.01 0.01]
[0.99 0.01 0.99]
[0.01 0.99 0.01]

[0.01 0.99 0.01]
[0.01 0.99 0.01]
[0.99 0.01 0.01 0.99]

[0.33 0.33 0.99 0.33 0.33 0.33 0.33 0.33 0.33]
[0.33 0.33 0.99]
[0.33 0.33 0.33]
[0.33 0.33 0.33]
  │()│}{
──┼──┼──
><│()│><
──┼──┼──
()│><│  

